# Tubes 2
## Feed Forward Neural Network
___Alvin Sullivan 13515048___

___Albertus Djauhari Djohan 13515054___

___Kevin 13515138___

### Membuat Sebuah Classifier

#### Multi Layer Neural Network
Classifier dibuat dengan sebuah kelas bernama `MultiLayerNN`. Kelas ini berfungsi untuk memodelkan neural network yang mampu melakukan pembelajaran dengan mini-batch stochastic gradient descent. Kelas ini memiliki atribut matriks data input tanpa label, matriks weight dari hidden node, matriks weight dari output node, banyak batch, konstanta learning rate, konstanta tolerance, konstanta momentum, dan banyak epochs. Kelas ini memiliki spesifikasi sebagai berikut.

- Jumlah hidden layer maksimal 10
- Jumlah node dalam setiap hidden layer dapat bervariasi
- Fully-connected layer
- Fungsi aktivasi berupa sigmoid untuk semua hidden layer maupun output layer
- Node output berjumlah 1
- Program memberikan pilihan untuk menggunakan momentum atau tidak
- Program mengimplementasikan mini-batch stochastic gradient descent

Kelas ini memiliki fungsi `train` untuk melakukan pembelajaran mini-batch stochastic gradient descent. Fungsi train melakukan pembelajaran sesuai dengan epochs dan batch yang ditentukan. Untuk setiap batch dalam epochs, dipanggil fungsi `gradient_descent` yang memanggil tiga fungsi lainnya secara berurutan sesuai algoritma gradient descent. Pertama dipanggil fungsi `feed_forward` untuk menentukan output setiap neuron. Kedua dipanggil fungsi `back_propagation` untuk menentukan delta setiap neuron. Ketiga dipanggil fungsi `update_weight` untuk mengubah weight setiap neuron sesuai dengan hasil dari fungsi-fungsi sebelumnya. Setelah seluruh epochs selesai, maka diperoleh sebuah model neural network dengan representasi matriks weight setiap neuron yang sudah diperbarui.

In [1]:
from __future__ import division
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.layers import Dropout
import keras

Using TensorFlow backend.


In [2]:
var_layer_num = 4
var_nodes = [4, 3 , 2, 3 ,1]
var_epoch = 10
var_batch = 2
var_momentum = 0.9
var_learning_rate = 1e-6

In [3]:
class MultiLayerNN:

    def __init__(self, data, hidden_node, output_node,
        num_batch, learning_rate_const, tolerance_const,
        **kwargs):
        
        self.instance = data
        self.w_hidden_node = hidden_node
        self.w_output_node = output_node
        # Gradient Descent Parameters
        self.batch_size = num_batch
        self.learning_rate = learning_rate_const
        self.tolerance = tolerance_const
        self.momentum = kwargs.get('momentum', 0)
        self.epochs = kwargs.get('epochs', 10)
       
    def train(self, instance_target):
        instance_target_t = np.array([instance_target]).T
        batch_iteration = int(np.ceil(self.instance.shape[0] / self.batch_size))
        old_loss = -np.inf
        for step in range(self.epochs):
            loss = 0
            for i in range(batch_iteration):
                start_index = i * self.batch_size
                end_index = i * self.batch_size + self.batch_size
                if end_index > len(instance_target_t):
                    end_index = len(instance_target_t)
                self.gradient_descent(self.instance[start_index:end_index:1], instance_target_t[start_index:end_index:1])
                loss = loss + self.loss_function(self.instance[start_index:end_index:1], instance_target_t[start_index:end_index:1])
        # Print Loss
            print ("Loss after epoch %i: %f" % (step, self.loss_function(self.instance, instance_target)))
            
            if np.abs(loss - old_loss) < self.tolerance:
                break
            old_loss = loss
    
    def feed_forward(self, instance):
        s = list()
        o = list()
        # Feed Forward Hidden Node
        s_temp = instance.dot(self.w_hidden_node[0].T)
        o_temp = self.sigmoid(s_temp)
        s.append(s_temp)
        o.append(o_temp)
        iteration = len(self.w_hidden_node)
        for i in range(1,iteration):
            s_temp = o[i-1].dot(self.w_hidden_node[i].T)
            o_temp = self.sigmoid(s_temp)
            o.append(o_temp)
        # Feed Forward Output Node
        s_out = o[-1].dot(self.w_output_node.T)
        o_out = self.sigmoid(s_out)
        return s, o, s_out, o_out

    def sigmoid(self, X):
        output = 1 / (1 + np.exp(-X))
        return np.matrix(output)

    def loss_function(self, instance, instance_target):
        _,_,_,o_out = self.feed_forward(instance)
        
        squared_error = np.square(instance_target - o_out)
        data_loss = np.sum(squared_error) / 2      
        return data_loss/len(instance)
    
    def back_propagation(self, instance_target, o, o_out):
        d = list()
        # Back Propagation Output Node
        d_temp = np.multiply(np.multiply(o_out, 1-o_out), instance_target-o_out)
        d.insert(0, d_temp)
        # Back Propagation Hidden Node
        d_temp = np.multiply(np.multiply(o[-1], 1-o[-1]), (self.w_output_node.T.dot(d[0].T)).T)
        d.insert(0, d_temp)
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            d_temp = np.multiply(np.multiply(o[i-1], 1-o[i-1]), (self.w_hidden_node[i].T.dot(d[0].T)).T)
            d.insert(0, d_temp)
        return d

    def update_weight(self, instance, o, d):
        # Update Weight Output Node
        self.w_output_node[0] = self.w_output_node[0] + self.momentum * self.w_output_node[0] + self.learning_rate * d[-1].T.dot(o[-1])
        # Update Weight Hidden Node
        iteration = len(self.w_hidden_node)
        for i in range(iteration-1, 0, -1):
            self.w_hidden_node[i] = self.w_hidden_node[i] + self.momentum * self.w_hidden_node[i] + self.learning_rate * d[i].T.dot(o[i-1])
        self.w_hidden_node[0] = self.w_hidden_node[0] + self.momentum * self.w_hidden_node[0] + self.learning_rate * d[0].T.dot(instance)

    def gradient_descent(self, instance, instance_target):
        # Feed Forward
        _,o,_,o_out = self.feed_forward(instance)
        # Back Propagation      
        d = self.back_propagation(instance_target, o, o_out)
        # Update Weight
        self.update_weight(instance, o, d)
    def predict(self, instance):
        _,_,_,o_out = self.feed_forward(instance)
#         return np.argmax(o_out,axis=1)
        return o_out

### Eksekusi Data Weather

Membaca dataset weather dan membentuk data train dan data test

In [4]:
file = "dataset/weather.csv"
data = pd.read_csv(file)
data = data.apply(preprocessing.LabelEncoder().fit_transform)
data = data.values
X = data[:, 1:]
y = data[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print('X_train')
print(X_train)

print('X_test')
print(X_test)

print('y_train')
print(y_train)

print('y_test')
print(y_test)

X_train
[[1 0 0 0]
 [1 1 0 1]
 [0 1 1 0]
 [0 1 0 1]
 [1 0 0 1]
 [1 0 1 0]
 [2 0 1 0]
 [0 1 0 1]
 [2 0 0 0]
 [2 1 1 1]
 [2 0 0 1]
 [0 1 1 1]]
X_test
[[2 1 0 1]
 [2 0 1 1]]
y_train
[2 0 1 2 0 2 1 1 2 2 1 0]
y_test
[1 0]


#### Classifier yang Dibuat

Inisialisasi matriks weight hidden node dan output node

In [5]:
np.random.seed(0)
w1 = np.random.randn(4, 4) / np.sqrt(4)
w2 = np.random.randn(3, 4) / np.sqrt(3)
w_output_node = np.random.randn(1, 3)
w_hidden_node = list()
w_hidden_node.append(w1)
w_hidden_node.append(w2)
print('Matriks weight hidden node:\n', w_hidden_node)
print('Matriks weight output node:\n', w_output_node)

Matriks weight hidden node:
 [array([[ 0.88202617,  0.2000786 ,  0.48936899,  1.1204466 ],
       [ 0.933779  , -0.48863894,  0.47504421, -0.0756786 ],
       [-0.05160943,  0.20529925,  0.07202179,  0.72713675],
       [ 0.38051886,  0.06083751,  0.22193162,  0.16683716]]), array([[ 0.86260696, -0.11844818,  0.18074972, -0.4931124 ],
       [-1.47396936,  0.37736687,  0.49908247, -0.42848917],
       [ 1.31044344, -0.83967841,  0.02641869, -0.10807065]])]
Matriks weight output node:
 [[1.53277921 1.46935877 0.15494743]]


Membuat model neural network

In [6]:
multiLayerNN = MultiLayerNN(X_train, w_hidden_node, w_output_node, var_batch, var_learning_rate, 1e-6, momentum = var_momentum, epochs = var_epoch)
multiLayerNN.train(y_train)
print('Matriks weight hidden node:\n', multiLayerNN.w_hidden_node)
print('Matriks weight output node:\n', multiLayerNN.w_output_node)

#x_test = np.matrix([[2,1,1,1],[1,0,0,1]])
y_test_res = multiLayerNN.predict(X_test)
print(y_test_res)

Loss after epoch 0: 4.000000
Loss after epoch 1: 4.000000
Loss after epoch 2: 4.000000
Matriks weight hidden node:
 [matrix([[ 91843.04813005,  20833.65496899,  50956.69646622,
         116669.13562695],
        [ 97231.93255169, -50880.67811945,  49465.09473308,
          -7880.21257501],
        [ -5373.95247589,  21377.26683314,   7499.43772988,
          75714.82323675],
        [ 39622.42057469,   6334.84869502,  23109.15111933,
          17372.31198483]]), matrix([[  89820.97795078,  -12333.69344049,   18820.99049921,
          -51346.4868605 ],
        [-153480.52264421,   39294.21375751,   51968.13599439,
          -44617.44141032],
        [ 136453.00357054,  -87433.48816727,    2750.90849769,
          -11253.10991707]])]
Matriks weight output node:
 [[159604.24115171 153000.43692523  16134.26770481]]
[[1.]
 [1.]]


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: RuntimeWarning: overflow encountered in exp


#### Classifier menggunakan keras

In [7]:
model = Sequential()

for i in range(0,len(var_nodes)):
    if i == 0:
        model.add(Dense(units=var_nodes[1], activation='sigmoid', input_dim=var_nodes[0]))
        model.add(Dropout(0.1))
    elif i == len(var_nodes)-1:
        model.add(Dense(units=1, activation='sigmoid'))
    else:
        model.add(Dense(units=var_nodes[i+1], activation='sigmoid'))
        model.add(Dropout(0.1))

In [8]:
sgd = optimizers.SGD(lr=0.01, decay=var_learning_rate, momentum=var_momentum, nesterov=True)

model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, batch_size=var_batch, nb_epoch=var_epoch, verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/10
12/12 [==============================] - 1s 52ms/step - loss: 1.0392 - acc: 0.2500
Epoch 2/10
12/12 [==============================] - 0s 2ms/step - loss: 0.8616 - acc: 0.3333
Epoch 3/10
12/12 [==============================] - 0s 2ms/step - loss: 0.5688 - acc: 0.3333
Epoch 4/10
12/12 [==============================] - 0s 2ms/step - loss: 0.2704 - acc: 0.3333
Epoch 5/10
12/12 [==============================] - 0s 2ms/step - loss: 0.1314 - acc: 0.3333
Epoch 6/10
12/12 [==============================] - 0s 2ms/step - loss: 0.0096 - acc: 0.3333 
Epoch 7/10
12/12 [==============================] - 0s 2ms/step - loss: -0.1132 - acc: 0.3333
Epoch 8/10
12/12 [==============================] - 0s 2ms/step - loss: -0.1711 - acc: 0.3333
Epoch 9/10
12/12 [==============================] - 0s 2ms/step - loss: -0.2435 - acc: 0.3333
Epoch 10/10
12/12 [==============================] - 0s 2ms/step - loss: -0.3040 - acc: 0.3333


In [10]:
score = model.evaluate(X_test, y_test, verbose=0)
print(score)

[1.306472897529602, 0.5]


### Pembagian Tugas
1. Alvin Sullivan - 13515048 - 
2. Albertus Djauhari - 13515054 - 
3. Kevin - 13515138 - 